In [1]:
import sys
sys.path.insert(0,'/cis/home/leebc/Projects/Mouse_Histology/code/pytorch/') # add code directory to path
# import lddmm
import torch_lddmm
# import numpy
import numpy as np
# import nibabel for i/o
import nibabel as nib
# import matplotlib for display
import matplotlib.pyplot as plt
import torch
import SimpleITK as sitk

In [8]:
def get_max_dimensions(files):
    file_dimensions = []
    file_spacing = []
    
    max1 = 0; max2 = 0;
    for file in files:
        # read file 
        print(file)
        itkimage = sitk.ReadImage(file)

        # get dimension
        dimension = itkimage.GetSize()
        spacing = itkimage.GetSpacing()
        file_dimensions.append(dimension)
        file_spacing.append(spacing)
        print(dimension, spacing)

        # get the maximum largest x and largest y dimensions of the imageset
        if dimension[0] > max1:
            max1 = dimension[0]
        if dimension[1] > max2:
            max2 = dimension[1]
            
    return max1, max2, file_dimensions, file_spacing

In [14]:
%%time 
import os 
filepath='/cis/project/auditory/deafcats/dat/hannover/'
os.chdir(filepath)

files = ['WK1D29_OT009.tif', 'WK1D29_OT014.tif', 'WK1D29_OT019.tif',
         'WK1D29_OT024.tif', 'WK1D29_OT029.tif', 'WK1D29_OT034.tif', 'WK1D29_OT039.tif',
         'WK1D29_OT044.tif', 'WK1D29_OT049.tif', 'WK1D29_OT054.tif', 'WK1D29_OT059.tif',
         'WK1D29_OT064.tif', 'WK1D29_OT069.tif', 'WK1D29_OT074.tif', 'WK1D29_OT079.tif',
         'WK1D29_OT084.tif', 'WK1D29_OT089.tif', 'WK1D29_OT094.tif', 'WK1D29_OT099.tif',]

# had problems reading files WK1D29_OT004, WK1D29_OT104, WK1D29_OT134, WK1D29_OT184
[max_x,max_y, file_dimensions, file_spacing] = get_max_dimensions(files)
size = [max_x, max_y, 1]

WK1D29_OT009.tif
(26659, 18708) (1.0, 1.0)
WK1D29_OT014.tif
(26717, 19438) (1.0, 1.0)
WK1D29_OT019.tif
(27649, 18719) (1.0, 1.0)
WK1D29_OT024.tif
(27618, 18738) (1.0, 1.0)
WK1D29_OT029.tif
(27673, 18668) (1.0, 1.0)
WK1D29_OT034.tif
(26674, 20107) (1.0, 1.0)
WK1D29_OT039.tif
(26668, 20854) (1.0, 1.0)
WK1D29_OT044.tif
(25714, 20132) (1.0, 1.0)
WK1D29_OT049.tif
(24730, 20138) (1.0, 1.0)
WK1D29_OT054.tif
(27603, 20112) (1.0, 1.0)
WK1D29_OT059.tif
(26663, 20154) (1.0, 1.0)
WK1D29_OT064.tif
(27607, 20867) (1.0, 1.0)
WK1D29_OT069.tif
(27611, 20849) (1.0, 1.0)
WK1D29_OT074.tif
(28567, 20845) (1.0, 1.0)
WK1D29_OT079.tif
(27561, 20131) (1.0, 1.0)
WK1D29_OT084.tif
(26668, 20139) (1.0, 1.0)
WK1D29_OT089.tif
(25723, 20107) (1.0, 1.0)
WK1D29_OT094.tif
(27588, 18706) (1.0, 1.0)
WK1D29_OT099.tif
(27583, 18715) (1.0, 1.0)
CPU times: user 16.8 s, sys: 37.2 s, total: 54 s
Wall time: 54 s


In [17]:
%%time
# Downsample images by factor of 0.5
factor = 0.125
size = [int(factor*max_x),int(factor*max_y)]
# Keep spacing the same

print(size)
for count,file in enumerate(files):
    print(count)
    # Read image from Hannover File Path in Cis Projects
    filepath='/cis/project/auditory/deafcats/dat/hannover/'
    os.chdir(filepath)
    img = sitk.ReadImage(file)
    spacing = (1/factor)*np.array(img.GetSpacing())
    resampled_image = sitk.Resample(img,size,sitk.AffineTransform(2),sitk.sitkLinear,[0,0], spacing)
    
    # Save resampled images to my home directory
    filepath = '/cis/home/jtrieu/Documents/HannoverNISSLResampled/'
#     os.makedirs(filepath)
    os.chdir(filepath)
    sitk.WriteImage(resampled_image, str(count)+'.tif')

    

[3570, 2608]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
CPU times: user 2min 46s, sys: 45.5 s, total: 3min 31s
Wall time: 1min 3s


***Convert to grayscale images*** 

In [19]:
from PIL import Image

# Change filepath to get to where files are saved
filepath = '/cis/home/jtrieu/Documents/HannoverNISSLResampled/'
os.chdir(filepath)

files = os.listdir(filepath)
os.makedirs('/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/')

for count,file in enumerate(files):
    print(count)
    filepath = '/cis/home/jtrieu/Documents/HannoverNISSLResampled/'
    os.chdir(filepath)
    img = sitk.ReadImage(file)
    img_arr = sitk.GetArrayFromImage(img)
    gray_sc = np.dot(img_arr,[0.299,0.587,0.144])
    gray_sc = gray_sc.astype(np.int8)
    gray_sc_im = sitk.GetImageFromArray(gray_sc)
    filepath = '/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/'
    os.chdir(filepath)
    sitk.WriteImage(gray_sc_im, str(count)+'gray.tif')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [23]:
%%time
import glob
# Change filepath to get to where files are saved
filepath = '/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/'
os.chdir(filepath)
files = glob.glob(filepath+"*.tif")

thresh = [(9,253),
          (9,253),
          (18,250),
          (16,253),
          (8,250),
          (11,250),
          (9,252),
          (9,251),
          (8,252),
          (9,252),
          (8,252),
          (8,252),
          (15,253),
          (9,253),
          (9,254),
          (10,255),
          (68,255),
          (58,255),
          (9,246)]
          
          
modified_images = []

for i, file in enumerate(files):
    print(file)
    filename = str(i) + 'gray.tif'
    img = sitk.ReadImage(filename)
    img = sitk.Cast(img+128, sitk.sitkFloat32)+128 # Fix issue with reading signed char tifs
    
    
    seg = sitk.BinaryThreshold(img,
                               lowerThreshold=thresh[i][0], upperThreshold=thresh[i][1],
                               insideValue=0, outsideValue=1)
    image_mask = sitk.Mask(img,seg,outsideValue = 0, maskingValue= 1)
        
    # Save image to modified images
    modified_images.append(image_mask)
    

filepath = '/cis/home/jtrieu/Documents'
os.chdir(filepath)
factor = 0.125
xy_spacing = 1.0745e-3 / factor
z_spacing = 250e-3
spacing = [xy_spacing, xy_spacing, z_spacing]
volume_images = sitk.JoinSeries(modified_images)
volume_images.SetSpacing(spacing)
sitk.WriteImage(volume_images, 'WK1D29_NISSL_01012020.img')


/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/0gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/1gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/2gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/3gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/4gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/5gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/6gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/7gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/8gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/9gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/10gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/11gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/12gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/13gray.tif
/cis/home/jtrieu/Documents/HannoverNISSLResampledGray/14gray.tif
/cis/home/jtrieu/Documents/Hannover